##### using CAPITAL LETTERS for sqlite syntax, table names, columns are in lower case
##### using ''' ''' where ever multiple line sqlite codes are used

In [1]:
import pandas as pd
import sqlite3

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/fivethirtyeight/data/master/airline-safety/airline-safety.csv")

##### will check the total number of rows in data, just to make sure data is not huge

In [3]:
len(df)

56

##### now we will connect & create a new sqlite database
* lets write that in a function so that we can just call it without rewriting it

In [4]:
def create_new_db(db_name):
    conn = sqlite3.connect(db_name + ".db")
    print "Database {} created successfully".format(db_name)
    return conn

##### check the columns in the data to help us create a new table

In [5]:
df.columns, len(df.columns)

(Index([u'airline', u'avail_seat_km_per_week', u'incidents_85_99',
        u'fatal_accidents_85_99', u'fatalities_85_99', u'incidents_00_14',
        u'fatal_accidents_00_14', u'fatalities_00_14'],
       dtype='object'), 8)

##### there are 8 columns and we are gonna create those in the table


In [6]:
create_query = '''
                CREATE TABLE IF NOT EXISTS airlines 
                    (airline TEXT NOT NULL,
                    avail_seat_km_per_week INT NOT NULL,
                    incidents_85_99 INT NOT NULL,
                    fatal_accidents_85_99 NOT NULL,
                    fatalities_85_99 INT NOT NULL,
                    incidents_00_14 NOT NULL,
                    fatal_accidents_00_14 INT NOT NULL,
                    fatalities_00_14 NOT NULL)
'''

In [7]:
con = create_new_db("airlines_safety")

Database airlines_safety created successfully


##### execute the create query statement

In [8]:
con.execute(create_query)

##### to check if the table is created, let's write some code which could help us for display purpose

In [9]:
def display_data(table_name):
    display_query = "SELECT * FROM {}".format(table_name)
    cursor = con.execute(display_query)
    rows = cursor.fetchall()
    for row in rows:
        print row
    return

##### insert the data by iterating the rows of df, this is not the best way to write to sqlite table, later will see a simple one line code which will do this 
##### with ease. Don't forget to commit

In [10]:
for ix, row in df.iterrows():
    insert_query = '''
                    INSERT OR IGNORE INTO airlines 
                        (airline, avail_seat_km_per_week, 
                         incidents_85_99, fatal_accidents_85_99,
                         fatalities_85_99, incidents_00_14,
                         fatal_accidents_00_14, fatalities_00_14)
                         VALUES (?, ?, ?, ?, ?, ?, ?, ?)                   
    
    '''
    nth_row = (row['airline'], row['avail_seat_km_per_week'],
         row['incidents_85_99'], row['fatal_accidents_85_99'],
         row['fatalities_85_99'], row['incidents_00_14'],
         row['fatal_accidents_00_14'], row['fatalities_00_14'])
    con.execute(insert_query, nth_row)
con.commit()

##### call the display function to how if records are inserted

In [11]:
display_data("airlines")

(u'Aer Lingus', 320906734, 2, 0, 0, 0, 0, 0)
(u'Aeroflot*', 1197672318, 76, 14, 128, 6, 1, 88)
(u'Aerolineas Argentinas', 385803648, 6, 0, 0, 1, 0, 0)
(u'Aeromexico*', 596871813, 3, 1, 64, 5, 0, 0)
(u'Air Canada', 1865253802, 2, 0, 0, 2, 0, 0)
(u'Air France', 3004002661L, 14, 4, 79, 6, 2, 337)
(u'Air India*', 869253552, 2, 1, 329, 4, 1, 158)
(u'Air New Zealand*', 710174817, 3, 0, 0, 5, 1, 7)
(u'Alaska Airlines*', 965346773, 5, 0, 0, 5, 1, 88)
(u'Alitalia', 698012498, 7, 2, 50, 4, 0, 0)
(u'All Nippon Airways', 1841234177, 3, 1, 1, 7, 0, 0)
(u'American*', 5228357340L, 21, 5, 101, 17, 3, 416)
(u'Austrian Airlines', 358239823, 1, 0, 0, 1, 0, 0)
(u'Avianca', 396922563, 5, 3, 323, 0, 0, 0)
(u'British Airways*', 3179760952L, 4, 0, 0, 6, 0, 0)
(u'Cathay Pacific*', 2582459303L, 0, 0, 0, 2, 0, 0)
(u'China Airlines', 813216487, 12, 6, 535, 2, 1, 225)
(u'Condor', 417982610, 2, 1, 16, 0, 0, 0)
(u'COPA', 550491507, 3, 1, 47, 0, 0, 0)
(u'Delta / Northwest*', 6525658894L, 24, 12, 407, 24, 2, 51)
(u'Eg

##### now lets do some stuff like deletion and updates
##### delete the row where airline name is "Gulf Air"

In [12]:
delete_query = " DELETE FROM airlines WHERE airline = 'Gulf Air' "
con.execute(delete_query)
con.commit()

In [13]:
display_data('airlines')

(u'Aer Lingus', 320906734, 2, 0, 0, 0, 0, 0)
(u'Aeroflot*', 1197672318, 76, 14, 128, 6, 1, 88)
(u'Aerolineas Argentinas', 385803648, 6, 0, 0, 1, 0, 0)
(u'Aeromexico*', 596871813, 3, 1, 64, 5, 0, 0)
(u'Air Canada', 1865253802, 2, 0, 0, 2, 0, 0)
(u'Air France', 3004002661L, 14, 4, 79, 6, 2, 337)
(u'Air India*', 869253552, 2, 1, 329, 4, 1, 158)
(u'Air New Zealand*', 710174817, 3, 0, 0, 5, 1, 7)
(u'Alaska Airlines*', 965346773, 5, 0, 0, 5, 1, 88)
(u'Alitalia', 698012498, 7, 2, 50, 4, 0, 0)
(u'All Nippon Airways', 1841234177, 3, 1, 1, 7, 0, 0)
(u'American*', 5228357340L, 21, 5, 101, 17, 3, 416)
(u'Austrian Airlines', 358239823, 1, 0, 0, 1, 0, 0)
(u'Avianca', 396922563, 5, 3, 323, 0, 0, 0)
(u'British Airways*', 3179760952L, 4, 0, 0, 6, 0, 0)
(u'Cathay Pacific*', 2582459303L, 0, 0, 0, 2, 0, 0)
(u'China Airlines', 813216487, 12, 6, 535, 2, 1, 225)
(u'Condor', 417982610, 2, 1, 16, 0, 0, 0)
(u'COPA', 550491507, 3, 1, 47, 0, 0, 0)
(u'Delta / Northwest*', 6525658894L, 24, 12, 407, 24, 2, 51)
(u'Eg

##### now do a update, replace airline name which has star to without star

In [14]:
update_query = "UPDATE airlines SET airline = 'SAS' WHERE airline = 'SAS*' "
con.execute(update_query)
con.commit()

In [15]:
display_data('airlines')

(u'Aer Lingus', 320906734, 2, 0, 0, 0, 0, 0)
(u'Aeroflot*', 1197672318, 76, 14, 128, 6, 1, 88)
(u'Aerolineas Argentinas', 385803648, 6, 0, 0, 1, 0, 0)
(u'Aeromexico*', 596871813, 3, 1, 64, 5, 0, 0)
(u'Air Canada', 1865253802, 2, 0, 0, 2, 0, 0)
(u'Air France', 3004002661L, 14, 4, 79, 6, 2, 337)
(u'Air India*', 869253552, 2, 1, 329, 4, 1, 158)
(u'Air New Zealand*', 710174817, 3, 0, 0, 5, 1, 7)
(u'Alaska Airlines*', 965346773, 5, 0, 0, 5, 1, 88)
(u'Alitalia', 698012498, 7, 2, 50, 4, 0, 0)
(u'All Nippon Airways', 1841234177, 3, 1, 1, 7, 0, 0)
(u'American*', 5228357340L, 21, 5, 101, 17, 3, 416)
(u'Austrian Airlines', 358239823, 1, 0, 0, 1, 0, 0)
(u'Avianca', 396922563, 5, 3, 323, 0, 0, 0)
(u'British Airways*', 3179760952L, 4, 0, 0, 6, 0, 0)
(u'Cathay Pacific*', 2582459303L, 0, 0, 0, 2, 0, 0)
(u'China Airlines', 813216487, 12, 6, 535, 2, 1, 225)
(u'Condor', 417982610, 2, 1, 16, 0, 0, 0)
(u'COPA', 550491507, 3, 1, 47, 0, 0, 0)
(u'Delta / Northwest*', 6525658894L, 24, 12, 407, 24, 2, 51)
(u'Eg

##### lets do some of these things in python way, using just the con object

In [16]:
df = pd.read_sql("SELECT * FROM airlines", con)

In [17]:
df.head()

,airline,avail_seat_km_per_week,incidents_85_99,fatal_accidents_85_99,fatalities_85_99,incidents_00_14,fatal_accidents_00_14,fatalities_00_14
0,Aer Lingus,320906734,2,0,0,0,0,0
1,Aeroflot*,1197672318,76,14,128,6,1,88
2,Aerolineas Argentinas,385803648,6,0,0,1,0,0
3,Aeromexico*,596871813,3,1,64,5,0,0
4,Air Canada,1865253802,2,0,0,2,0,0


In [18]:
df_delete = pd.read_sql("SELECT * FROM airlines WHERE airline != 'Air Canada'", con)

In [19]:
len(df)

55

In [20]:
len(df_delete)

54

##### will write the data frame result to the table in a single line

In [21]:
df_delete.to_sql("airline", con, if_exists="replace")

##### so we can read in the sqlite table using pandas dataframe, then do all the stuff in pandas and write it back to sqlite (without getting to write too much of sqlite code)

##### todos: explore groupy with sqlite